## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-04-06-53-00 +0000,wsj,Swiss Inflation Holds Steady as SNB Mulls Rate...,https://www.wsj.com/economy/central-banking/sw...
1,2025-09-04-06-50-14 +0000,bbc,"Rayner at risk of fine over stamp duty, tax ex...",https://www.bbc.com/news/articles/c62n366q306o...
2,2025-09-04-06-50-00 +0000,wsj,The Child Among Autocrats: Kim Jong Un’s Young...,https://www.wsj.com/world/asia/north-korea-kim...
3,2025-09-04-06-48-22 +0000,nypost,"Man drove across country with weapons, made th...",https://nypost.com/2025/09/04/us-news/alabama-...
4,2025-09-04-06-38-14 +0000,bbc,Allies ready to support Ukraine before and aft...,https://www.bbc.com/news/articles/c5y85x58nj4o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2358/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
57,trump,47
26,china,21
47,putin,17
876,new,16
289,up,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
222,2025-09-03-16-36-24 +0000,nypost,"Trump says Putin, Xi were just putting on a sh...",https://nypost.com/2025/09/03/us-news/trump-sa...,127
164,2025-09-03-19-50-17 +0000,nypost,Trump rips reporter for saying he’s taken ‘no ...,https://nypost.com/2025/09/03/us-news/trump-ri...,112
39,2025-09-04-02-55-30 +0000,nypost,Trump admin demands Supreme Court rule quickly...,https://nypost.com/2025/09/03/us-news/trump-ad...,109
35,2025-09-04-03-24-20 +0000,nyt,Xi Parades Military Strength as Trump Accuses ...,https://www.nytimes.com/2025/09/02/world/asia/...,109
330,2025-09-03-08-26-24 +0000,nyt,"China’s Military Parade, in Photos: Xi Unveils...",https://www.nytimes.com/2025/09/03/world/asia/...,109


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
222,127,2025-09-03-16-36-24 +0000,nypost,"Trump says Putin, Xi were just putting on a sh...",https://nypost.com/2025/09/03/us-news/trump-sa...
260,67,2025-09-03-14-32-35 +0000,nypost,Kim Jong Un is seen with his young daughter in...,https://nypost.com/2025/09/03/world-news/kim-j...
39,62,2025-09-04-02-55-30 +0000,nypost,Trump admin demands Supreme Court rule quickly...,https://nypost.com/2025/09/03/us-news/trump-ad...
61,60,2025-09-04-01-00-00 +0000,wsj,President Trump’s campaign to bring the Federa...,https://www.wsj.com/economy/central-banking/tr...
110,53,2025-09-03-22-33-42 +0000,nypost,JD and Usha Vance pay their respects to victim...,https://nypost.com/2025/09/03/us-news/jd-and-u...
176,49,2025-09-03-19-24-00 +0000,wsj,New Orleans might be the next city to get addi...,https://www.wsj.com/politics/policy/trump-mull...
82,47,2025-09-04-00-00-00 +0000,wsj,Hundreds of current and former Health and Huma...,https://www.wsj.com/politics/policy/rfk-jr-cdc...
212,44,2025-09-03-17-16-35 +0000,nyt,"After Graham Linehan’s Arrest, Police Chief Sa...",https://www.nytimes.com/2025/09/03/world/europ...
62,41,2025-09-04-01-00-00 +0000,wsj,Google Ruling Shows Antitrust Tools Struggle t...,https://www.wsj.com/us-news/law/google-search-...
119,36,2025-09-03-22-07-00 +0000,wsj,Florida is proposing to eliminate all vaccine ...,https://www.wsj.com/health/florida-seeks-to-en...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
